In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from pytorch_lightning.callbacks import Callback
import pandas as pd
import pickle
from pytorch_lightning.callbacks import ModelCheckpoint

D:\Anaconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [42]:
users = pd.read_csv('processed_dataset/MovieLens-1M/users/users_movielens.csv')
movies = pd.read_csv('processed_dataset/MovieLens-1M/movies/movies_movielens.csv')
full_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_full_movielens.csv')
train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_train_movielens.csv')
val_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_val_movielens.csv')
test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_test_movielens.csv')

In [43]:
def generate_user_texts_with_history(users, movies, ratings):
    user_histories = {user_id: [] for user_id in users['user_id'].unique()}
    user_texts = []

    # Convert relevant columns to dictionaries for faster access
    user_features_dict = users.set_index('user_id').to_dict('index')
    movie_titles_dict = movies.set_index('item_id')['genres'].to_dict()

    for _, row in ratings.iterrows():
        user_id = row['user_id']
        movie_id = row['item_id']

        # Get user features
        user = user_features_dict[user_id]
        user_features = f"occupation: {user['occupation']} [SEP] gender: {user['gender']}"
        # user_features = f"occupation: {user['occupation']} [SEP] gender: {user['gender']}"

        # Append the user's history (only the last 3 movies)
        history_movies = [movie_titles_dict[mid] for mid in user_histories[user_id][-10:]]
        history_str = ", ".join(history_movies)

        # Combine user features and history
        # if history_str:
        #     combined_features = f"{user_features} [SEP] genres: {history_str}"
        # else:
        #     combined_features = f"{user_features}"
        # if history_str:
        combined_features = f"genres: {history_str}"
        # else:
        #     combined_features = f"{user_features}"
        user_texts.append(combined_features)

        # Update the user history after generating combined features
        user_histories[user_id].append(movie_id)

    return user_texts


In [44]:
def generate_last_user_texts_with_history(users, movies, ratings):
    user_histories = {user_id: [] for user_id in users['user_id'].unique()}
    last_user_texts = {}

    # Convert relevant columns to dictionaries for faster access
    user_features_dict = users.set_index('user_id').to_dict('index')
    movie_titles_dict = movies.set_index('item_id')['genres'].to_dict()

    for _, row in ratings.iterrows():
        user_id = row['user_id']
        movie_id = row['item_id']

        # Get user features
        user = user_features_dict[user_id]
        user_features = f"occupation: {user['occupation']} [SEP] gender: {user['gender']}"
        # user_features = f"occupation: {user['occupation']} [SEP] gender: {user['gender']}"

        # Append the user's history (only the last 3 movies)
        history_movies = [movie_titles_dict[mid] for mid in user_histories[user_id][-10:]]
        history_str = ", ".join(history_movies)

        # Combine user features and history
        # if history_str:
        #     combined_features = f"{user_features} [SEP] genres: {history_str}"
        # else:
        #     combined_features = f"{user_features}"

        combined_features = f"genres: {history_str}"


        # Update the dictionary to keep the last text for each user
        last_user_texts[user_id] = combined_features

        # Update the user history after generating combined features
        user_histories[user_id].append(movie_id)

    return last_user_texts

# Generate the last user texts for the validation data
val_last_user_texts = generate_last_user_texts_with_history(users, movies, val_ratings)

In [24]:
movies

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [46]:
train_user_texts = generate_user_texts_with_history(users, movies, train_ratings)
val_user_texts = generate_user_texts_with_history(users, movies, val_ratings)
test_user_texts = generate_user_texts_with_history(users, movies, test_ratings)

In [45]:
print(val_last_user_texts.get(10))

genres: Action|Children's|Fantasy, Children's|Drama|Fantasy, Comedy, Comedy|Romance, Comedy, Comedy|Drama, Animation|Comedy, Comedy|Drama, Comedy, Animation|Comedy|Thriller


In [47]:
print(train_user_texts[7])

genres: Drama, Animation|Children's|Musical, Drama|Romance, Comedy|Sci-Fi, Romance, Drama, Drama


In [15]:
# # Save user embeddings locally
# with open('train_user_texts.pkl', 'wb') as f:
#     pickle.dump(train_user_texts, f)
#
# print("Train user embeddings saved successfully.")
#
# with open('val_user_texts.pkl', 'wb') as f:
#     pickle.dump(val_user_texts, f)
#
# print("Validation user embeddings saved successfully.")
#
# with open('test_user_texts.pkl', 'wb') as f:
#     pickle.dump(test_user_texts, f)
#
# print("Test user embeddings saved successfully.")

In [16]:
# # Load user texts from file
# with open('./text_for_embeddings/last_three_history/train_user_texts.pkl', 'rb') as f:
#     train_user_texts = pickle.load(f)
# print("Train user text loaded successfully.")
#
# with open('./text_for_embeddings/last_three_history/val_user_texts.pkl', 'rb') as f:
#     val_user_texts = pickle.load(f)
# print("Validation user text loaded successfully.")
#
# with open('./text_for_embeddings/last_three_history/test_user_texts.pkl', 'rb') as f:
#     test_user_texts = pickle.load(f)
# print("Test user text loaded successfully.")

In [73]:
# Combine movie features into a single string for each movie
movies['movie_features'] = 'title: ' + movies['title'] + ' [SEP] genres: ' + movies['genres']
# movies['movie_features'] = 'genres: ' + movies['genres']


In [74]:
movies['movie_features'][589]

'title: Silence of the Lambs, The (1991) [SEP] genres: Drama|Thriller'

In [75]:
# Create a dictionary for fast lookup
movie_features_dict = movies.set_index('item_id')['movie_features'].to_dict()

# Create lists of user and item texts
item_texts = [movie_features_dict[movieId] for movieId in full_ratings['item_id'].unique()]

# Create a mapping from userId and movieId to indices
movie_id_to_idx = {movieId: idx for idx, movieId in enumerate(full_ratings['item_id'].unique())}

# Map userId and movieId in ratings_df to indices
train_ratings['movie_idx'] = train_ratings['item_id'].map(movie_id_to_idx)

# Map userId and movieId in ratings_val to indices
val_ratings['movie_idx'] = val_ratings['item_id'].map(movie_id_to_idx)

# Map userId and movieId in ratings_test to indices
test_ratings['movie_idx'] = test_ratings['item_id'].map(movie_id_to_idx)

# Extract user indices, item indices, and ratings
train_item_indices = torch.LongTensor(train_ratings['movie_idx'].values).to(device)
train_labels = torch.FloatTensor(train_ratings['rating'].values).to(device)

# Extract user indices, item indices, and ratings for validation
val_item_indices = torch.LongTensor(val_ratings['movie_idx'].values).to(device)
val_labels = torch.FloatTensor(val_ratings['rating'].values).to(device)

# Extract user indices, item indices, and ratings for test
test_item_indices = torch.LongTensor(test_ratings['movie_idx'].values).to(device)
test_labels = torch.FloatTensor(test_ratings['rating'].values).to(device)


In [51]:
test_ratings

,user_id,item_id,rating,timestamp,movie_idx
0,1,783,4,978824291,154
1,1,2294,4,978824291,1330
2,1,2355,5,978824291,669
3,1,1907,4,978824330,937
4,1,1566,4,978824330,1066
...,...,...,...,...,...
102754,6040,2917,4,997454429,1547
102755,6040,1784,3,997454464,538
102756,6040,1921,4,997454464,946
102757,6040,161,3,997454486,1856


In [52]:
from torch.utils.data import Dataset, DataLoader

class CustomTextDataset(Dataset):
    def __init__(self, users, item_ids, ratings):
        self.users = users
        self.item_ids = item_ids
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        users = self.users[idx]
        item_id = self.item_ids[idx]
        rating = self.ratings[idx]
        return users, item_id, rating

In [53]:
# Create DataLoader for training data
train_dataset = CustomTextDataset(train_user_texts, train_item_indices, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

# Create DataLoader for validation data
val_dataset = CustomTextDataset(val_user_texts, val_item_indices, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True, drop_last=True)

# Create DataLoader for test data
test_dataset = CustomTextDataset(test_user_texts, test_item_indices, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, drop_last=True)

In [35]:
'occupation: doctor/health care [SEP] age: 25-34 [SEP] gender: Male [SEP] positively rated movies: ... [SEP] negatively rated movies: ...'

'occupation: doctor/health care [SEP] age: 25-34 [SEP] gender: Male [SEP] positively rated movies: ... [SEP] negatively rated movies: ...'

In [54]:
class TwoTowerModel(pl.LightningModule):
    def __init__(self, user_model_name, item_model_name, embedding_size=384):
        super(TwoTowerModel, self).__init__()
        self.user_model = SentenceTransformer(user_model_name)
        self.item_model = SentenceTransformer(item_model_name)

        self.user_fc = nn.Linear(embedding_size, embedding_size)
        self.item_fc = nn.Linear(embedding_size, embedding_size)

        self.criterion = nn.MSELoss()
        self.epoch_losses = {'train_loss': [], 'val_loss': []}

    def forward(self, user_text, item_text):
        user_embedding = self.user_model.encode(user_text, convert_to_tensor=True).to(device)
        item_embedding = self.item_model.encode(item_text, convert_to_tensor=True).to(device)

        user_output = self.user_fc(user_embedding)
        item_output = self.item_fc(item_embedding)

        dot_product = torch.matmul(user_output.squeeze(), item_output.T)
        dot_product = 4 * torch.sigmoid(dot_product) + 1

        return dot_product

    def training_step(self, batch, batch_idx):
        users, items, ratings = batch

        items = [item_texts[i] for i in items.tolist()]

        preds = self(users, items)

        loss = self.criterion(preds, ratings)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        users, items, ratings = batch

        items = [item_texts[i] for i in items.tolist()]

        preds = self(users, items)

        loss = self.criterion(preds, ratings)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-5)

class PrintLossesCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get('train_loss')
        if train_loss is not None:
            pl_module.epoch_losses['train_loss'].append(train_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Train Loss: {train_loss.item()}")

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics.get('val_loss')
        if val_loss is not None:
            pl_module.epoch_losses['val_loss'].append(val_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Val Loss: {val_loss.item()}")

In [55]:
# model = TwoTowerModel(user_model_name='paraphrase-MiniLM-L6-v2', item_model_name='paraphrase-MiniLM-L6-v2')
model = TwoTowerModel(user_model_name='paraphrase-MiniLM-L12-v2', item_model_name='paraphrase-MiniLM-L12-v2')

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Metric to monitor
    dirpath='checkpoints/',  # Directory to save the checkpoints
    filename='with-history-best-checkpoint',  # Filename for the best model
    save_top_k=1,  # Save only the top 1 model
    mode='min'  # Mode to save the best model (min for validation loss)
)

trainer = pl.Trainer(max_epochs=5, log_every_n_steps=1, callbacks=[PrintLossesCallback()], enable_progress_bar=True)
trainer.fit(model, train_dataloader, val_dataloader)

# Print losses after training completes
print("Epoch losses:")
for epoch in range(trainer.max_epochs):
    train_loss = model.epoch_losses['train_loss'][epoch] if epoch < len(model.epoch_losses['train_loss']) else 'N/A'
    val_loss = model.epoch_losses['val_loss'][epoch] if epoch < len(model.epoch_losses['val_loss']) else 'N/A'
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}")

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params | Mode 
-----------------------------------------------------------
0 | user_model | SentenceTransformer | 33.4 M | train
1 | item_model | SentenceTransformer | 33.4 M | train
2 | user_fc    | Linear              | 147 K  | train
3 | item_fc    | Linear              | 147 K  | train
4 | criterion  | MSELoss             | 0      | train
-----------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
268.063   Total estimated model params siz

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:475: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  4.72it/s]Epoch 1: Val Loss: 1.4957122802734375
                                                                           

D:\Anaconda\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 12464/12464 [45:21<00:00,  4.58it/s, v_num=3]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [06:13<00:00,  4.17it/s]Epoch 1: Val Loss: 1.2435319423675537

Epoch 1: 100%|██████████| 12464/12464 [44:16<00:00,  4.69it/s, v_num=3]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [07:56<00:00,  3.26it/s]Epoch 2: Val Loss: 1.230196237564087

Epoch 2: 100%|██████████| 12464/12464 [53:17<00:00,  3.90it/s, v_num=3] 
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [05:13<00:00,  4.97it/s]Epoch 3: Val Loss: 1.2221845388412476

Epoch 3: 100%|██████████| 12464/12464 [1:00:33<00:00,  3.43it/s, v_num=3]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [05:54<00:00,  4.39it/s]Epoch 4: Val Loss: 1.2250391244888306

Epoch 4: 100%|██████████| 12464/12464 [38:17<00:00,  5.43it/s, v_n

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 12464/12464 [42:25<00:00,  4.90it/s, v_num=3]
Epoch losses:
Epoch 1: Train Loss: 1.1066803932189941, Val Loss: 1.4957122802734375
Epoch 2: Train Loss: 1.2313159704208374, Val Loss: 1.2435319423675537
Epoch 3: Train Loss: 1.126348614692688, Val Loss: 1.230196237564087
Epoch 4: Train Loss: 0.8742800951004028, Val Loss: 1.2221845388412476
Epoch 5: Train Loss: 0.9683778285980225, Val Loss: 1.2250391244888306


In [20]:
model.epoch_losses

{'train_loss': [0.9391066431999207,
  1.0028347969055176,
  0.9369568824768066,
  0.9828064441680908,
  0.9409568905830383],
 'val_loss': [10.464865684509277,
  1.1707780361175537,
  1.138741135597229,
  1.120898962020874,
  1.1324385404586792,
  1.116618275642395]}

# Evaluation

In [56]:
# Assuming the training part has been done already, load the best model checkpoint

# best_model_path = './lightning_logs/paraphrase-MiniLM-L6-v2/not-binarized/history_5-epochs_lr-1e-5/checkpoints/epoch=4-step=93765.ckpt'  # Path where the best model is saved
# best_model_path = './lightning_logs/version_0/checkpoints/epoch=0-step=12464.ckpt'  # Path where the best model is saved
# best_model_path = './lightning_logs/paraphrase-MiniLM-L12-v2/not-binarized/history_5-epochs_lr-1e-5_(occu + gen ) (new format - only genre for movies) (no header tag)/checkpoints/epoch=4-step=62320.ckpt'  # Path where the best model is saved
best_model_path = './lightning_logs/version_3/checkpoints/epoch=4-step=62320.ckpt'
# best_model = TwoTowerModel.load_from_checkpoint(best_model_path, user_model_name='paraphrase-MiniLM-L6-v2', item_model_name='paraphrase-MiniLM-L6-v2').to(device)
best_model = TwoTowerModel.load_from_checkpoint(best_model_path, user_model_name='paraphrase-MiniLM-L12-v2', item_model_name='paraphrase-MiniLM-L12-v2').to(device)


D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Calculations

In [79]:
def get_top_n_items_without_history_unseen_items(model, userId, n):
    # Ensure the model is in evaluation mode
    model.eval()

    # Get the user text for the given userId
    user_text = val_last_user_texts[userId]

    # Encode the user text
    user_embedding = model.user_model.encode(user_text, convert_to_tensor=True).to(device)

    # Compute the scores (dot product between user embedding and each item embedding)
    user_output = model.user_fc(user_embedding).to(device)
    item_output = model.item_fc(full_items_embeddings).to(device)
    dot_product = torch.matmul(user_output, item_output.t()).squeeze()

    # Get items the user has seen in the training and validation data
    seen_items_train = train_ratings[train_ratings['user_id'] == userId]['item_id'].values
    seen_items_val = val_ratings[val_ratings['user_id'] == userId]['item_id'].values
    seen_items = set(np.concatenate((seen_items_train, seen_items_val)))

    # Get the top n + len(seen_items) item indices and their scores
    top_n_scores, top_n_indices = torch.topk(dot_product, n + len(seen_items))

    # Map indices back to item IDs
    top_n_item_ids = [list(movie_id_to_idx.keys())[list(movie_id_to_idx.values()).index(idx.item())] for idx in top_n_indices]

    # Filter out seen items
    unseen_top_n_item_ids = [item for item in top_n_item_ids if item not in seen_items]

    return unseen_top_n_item_ids[:n]

In [76]:
# Assuming full_items_embeddings is already defined
full_items_embeddings = torch.stack([best_model.item_model.encode(item_text, convert_to_tensor=True) for item_text in item_texts]).to(device)

In [77]:
item_texts[8]

'title: Ciao, Professore! (Io speriamo che me la cavo ) (1993) [SEP] genres: Drama'

In [78]:
full_items_embeddings[8]

tensor([-2.2607e-01, -1.4285e-01,  1.0240e-01,  3.3239e-02, -2.5826e-01,
         2.7096e-01,  1.9785e-01,  5.2252e-02,  9.6593e-02,  1.9839e-01,
         1.6833e-01,  5.7473e-02, -4.7475e-02,  2.8626e-01, -4.2003e-01,
        -1.9080e-01,  1.8600e-01,  2.6678e-01,  2.1197e-01,  8.7586e-02,
         2.1185e-01, -3.0451e-02,  1.1191e-01, -2.0692e-01, -1.3530e-01,
        -3.2118e-01, -1.0356e-01,  1.5984e-01, -1.6035e-01, -4.2185e-01,
        -1.6459e-01, -4.2135e-02,  3.3600e-01,  2.1103e-01, -3.3594e-01,
         2.5755e-01, -2.1724e-01,  5.0386e-02,  1.9934e-01,  1.2678e-01,
        -1.6753e-01,  2.4814e-01,  1.2218e-02,  2.7016e-01, -1.9260e-01,
        -3.0105e-01,  1.0088e-02, -2.3721e-02, -2.4030e-01,  1.3153e-01,
        -3.7549e-01,  2.2445e-01, -2.8223e-01,  3.3530e-02, -2.0871e-01,
        -2.7440e-01,  3.9110e-01,  2.9165e-01,  3.5558e-01,  1.4777e-01,
         1.4917e-01,  1.9578e-01, -4.3344e-01,  1.0387e-01, -8.9261e-03,
        -9.3977e-02, -2.7168e-02,  2.6888e-02, -2.7

## Type 0

In [80]:
def dcg(scores, k):
    scores = np.asfarray(scores)[:k]
    return np.sum(scores / np.log2(np.arange(2, scores.size + 2)))

def ndcg_at_k(labels, k):
    ideal_labels = sorted(labels, reverse=True)
    return dcg(labels, k) / dcg(ideal_labels, k)

def recall_at_k(labels, relevant_count, k):
    return np.sum(labels[:k]) / relevant_count

def mrr_at_k(labels, k):
    for i, label in enumerate(labels[:k]):
        if label == 1:
            return 1 / (i + 1)
    return 0

def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []
    recall_scores = []
    mrr_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)
        # print(recommended_items)
        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values
        print(user)
        y_score = [1 if item in test_items else 0 for item in recommended_items]

        ndcg = ndcg_at_k(y_score, k)
        recall = recall_at_k(y_score, len(test_items), k)
        mrr = mrr_at_k(y_score, k)

        ndcg_scores.append(ndcg)
        recall_scores.append(recall)
        mrr_scores.append(mrr)

    avg_ndcg = np.nanmean(ndcg_scores)
    avg_recall = np.nanmean(recall_scores)
    avg_mrr = np.nanmean(mrr_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg,
        'Recall@{}'.format(k): avg_recall,
        'MRR@{}'.format(k): avg_mrr,
    }


all_items = movies['item_id'].unique()
# Evaluate the model
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=10)
print(eval_result)

1
2
3
4
5
6


C:\Users\Hooman\AppData\Local\Temp\ipykernel_26392\1211446300.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return dcg(labels, k) / dcg(ideal_labels, k)


7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280


In [ ]:
{'NDCG@5': 0.8654867659410482}

## Type 2

In [22]:
def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)

        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values

        y_score = [
            user_test_data[user_test_data['item_id'] == item]['rating'].values[0] if item in test_items else 2.5
            for item in recommended_items
        ]

        ndcg = ndcg_at_k(y_score, k)
        ndcg_scores.append(ndcg)

    avg_ndcg = np.nanmean(ndcg_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg
    }

all_items = movies['item_id'].unique()
# Evaluate the model
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Type 3

In [23]:
def dcg(scores, k):
    scores = np.asfarray(scores)[:k]
    return np.sum(scores / np.log2(np.arange(2, scores.size + 2)))

def ndcg_at_k(labels, k):
    ideal_labels = sorted(labels, reverse=True)
    return dcg(labels, k) / dcg(ideal_labels, k)

def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)

        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values
        # print(user)

        y_score = [
            user_test_data[user_test_data['item_id'] == item]['rating'].values[0] if item in test_items else 0
            for item in recommended_items
        ]

        ndcg = ndcg_at_k(y_score, k)
        ndcg_scores.append(ndcg)

    avg_ndcg = np.nanmean(ndcg_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg
    }

all_items = movies['item_id'].unique()
# Evaluate the model
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)

1
2
3
4
5
6
7
8


C:\Users\Hooman\AppData\Local\Temp\ipykernel_4736\4020610523.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return dcg(labels, k) / dcg(ideal_labels, k)


9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281


## Type 1

In [ ]:
def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)

        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values
        print(user)
        # y_score = [
        #     user_test_data[user_test_data['item_id'] == item]['rating'].values[0] if item in test_items else 0
        #     for item in recommended_items
        # ]
        y_score = [
            1 if (item in test_items and user_test_data[user_test_data['item_id'] == item]['label'].values[0] == 1) else 0
            for item in recommended_items
        ]
        # y_score = [
        #     1 if (item in test_items and user_test_data[user_test_data['item'] == item]['label'].values[0] == 1) else 0
        #     for item in recommended_items
        # ]

        ndcg = ndcg_at_k(y_score, k)
        ndcg_scores.append(ndcg)

    # avg_ndcg = np.mean(np.nan_to_num(ndcg_scores, nan=0.0))
    avg_ndcg = np.nanmean(ndcg_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg
    }

all_items = movies['item_id'].unique()
# Evaluate the model
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)